# Final Project

In [1]:
import psycopg2

In [2]:
database = '_adventureworkslt'
host = 'localhost'
credentials = [
    ('ivan', 'sql'),
    ('kirill', 'sql'),
    ('sophie', 'sql')
]

In [61]:


def start_planning(year, quarter, user, pwd, database):
    """ 
    Task steps
    1. Delete plan data from the plan_data table related to the target year and quarter.
    2. Delete all records related to the target quarter from the plan_status table.
    3. Create the necessary planning status records in the plan_status table for the selected quarter.
    4. Generate a version N of planning data in the plan_data table. 
    5. Copy the data from the version N and insert it further to the plan_data table changing the version to P.
    6. Store the name of the user who called the function, in the records of the plan_status table.
    """

    # Supportive queries
    qry_plan_data_cleanup = """
        delete from plan_data 
        where quarterid = %s;
        """

    qry_plan_status_cleanup = """
        delete from plan_status 
        where quarterid = %s;
        """
    
    qry_plan_status_fillin = """
        insert into plan_status (quarterid, status, country)
        select %s, 'R', countrycode
        from country2 c;
        """

    qry_plan_data_fillin_N = """
        insert into plan_data
        with 
        quarterly_sales as (
            select distinct c.countrycode,
                cs.categoryid as pcid,
                cs.qr,
                sum(cs.salesamt) over (partition by c.countrycode, cs.categoryid, cs.qr) as salesamt
            from company_sales as cs
                join company as c on c.id = cs.cid 
            where cs.ccls in ('A', 'B') and cs.year in (%s - 1, %s - 2) and cs.quarter_yr = %s
        ),
        average_sales as (
            select distinct countrycode, 
                pcid,
                avg(salesamt) over (partition by countrycode, pcid) as salesamt
            from quarterly_sales
        ),
        plan_template as (
            select distinct c2.countrycode,
                p2.pcid
            from product2 as p2
                cross join country2 as c2
        )
        select 'N' as versionid, 
            pt.countrycode as country,
            %s as quarterid,
            pt.pcid,
            coalesce (avs.salesamt, 0) as salesamt
        from plan_template as pt
        left join average_sales as avs using(countrycode, pcid);
        """
    
    qry_plan_data_fillin_P = """
        insert into plan_data
        select 'P' as versionid,
            country,
            quarterid,
            pcid,
            salesamt
        from plan_data
        where versionid = 'N' and quarterid = %s;
        """

    qry_plan_status_update_author = """
        update plan_status
        set author = current_user
        where quarterid = %s;
        """

    # Setting the connection
    con = psycopg2.connect(database=database, user=user, password=pwd, host='localhost')
    cur = con.cursor()

    # Executing queries
    qr = f'{year}.{quarter}'
    try:
        cur.execute(qry_plan_data_cleanup, (qr,))
        cur.execute(qry_plan_status_cleanup, (qr,))
        cur.execute(qry_plan_status_fillin, (qr,))
        cur.execute(qry_plan_data_fillin_N, (year, year, quarter, qr))
        cur.execute(qry_plan_data_fillin_P, (qr,))
        cur.execute(qry_plan_status_update_author, (qr,))
        con.commit()
    except Exception as err:
        con.rollback()
        print(err)
        
    con.close()

In [63]:
def set_lock(year, quarter, user, pwd, database):

    # Supportive queries
    qry_plan_status_lock = """
        update plan_status as ps
        set status = %s,
            modifieddatetime = current_timestamp,
            author = current_user
        from country_managers as cm
        where ps.country = cm.country
        and status = %s
        and ps.quarterid = %s
        and cm.username = current_user;
        """
    
    # Setting the connection
    con = psycopg2.connect(database=database, user=user, password=pwd, host='localhost')
    cur = con.cursor()

    # Executing queries
    qr = f'{year}.{quarter}'
    try:
        cur.execute(qry_plan_status_lock, ('L', 'R', qr))
        con.commit()
    except Exception as err:
        con.rollback()
        print(err)
        
    con.close() 

In [64]:
def remove_lock(year, quarter, user, pwd, database):

    # Supportive queries
    qry_plan_status_unlock = """
        update plan_status as ps
        set status = %s,
            modifieddatetime = current_timestamp,
            author = current_user
        from country_managers as cm
        where ps.country = cm.country
        and status = %s
        and ps.quarterid = %s
        and cm.username = current_user
        and ps.author = current_user;
        """

    # Setting the connection
    con = psycopg2.connect(database=database, user=user, password=pwd, host='localhost')
    cur = con.cursor()

    # Executing queries
    qr = f'{year}.{quarter}'
    try:
        cur.execute(qry_plan_status_unlock, ('R', 'L', qr))
        con.commit()
    except Exception as err:
        con.rollback()
        print(err)
        
    con.close()  

In [69]:
def accept_plan(year, quarter, user, pwd, database):
    """
    Processing steps
    1. Clear the A version of plan data for specific quarter and countries accessible to the current user
    2. Read data available to the current user from the version P and save its copy as the version A
    3. Change the status of the processed from 'R' to 'A'
    4. When updating the status, also save a timestamp in the modifiedtimestamp column.
    """

    # Supportive queries
    qry_plan_data_cleanup_A = """
        delete 
        from plan_data as pd 
            using country_managers as cm
        where pd.country = cm.country
            and pd.quarterid = %s
            and pd.versionid = 'A'
            and cm.username = current_user;
        """
    
    qry_plan_data_fillin_A = """
        insert into plan_data   
        select 'A' as versionid, 
            country, 
            quarterid, 
            pcid, 
            salesamt 
        from plan_data as pd
        join plan_status as ps using (quarterid, country)
        join country_managers as cm using (country)
        where pd.quarterid = %s and pd.versionid = 'P' 
        and status = 'R' and cm.username = current_user;
        """

    qry_plan_status_update_A = """
        update plan_status as ps
        set status = %s,
            modifieddatetime = current_timestamp,
            author = current_user
        from country_managers as cm
        where ps.country = cm.country
        and status = %s
        and ps.quarterid = %s
        and cm.username = current_user
        and ps.author = current_user;
        """

    # Setting the connection
    con = psycopg2.connect(database=database, user=user, password=pwd, host='localhost')
    cur = con.cursor()

    # Executing queries
    qr = f'{year}.{quarter}'
    try:
        cur.execute(qry_plan_data_cleanup_A, (qr,))
        cur.execute(qry_plan_data_fillin_A, (qr,))
        cur.execute(qry_plan_status_update_A, ('A', 'R', qr))
        con.commit()
    except Exception as err:
        con.rollback()
        print(err)
        
    con.close()

In [ ]:
start_planning(2014, 1, 'ivan', 'sql', '_adventureworkslt')

In [ ]:
set_lock(2014, 1, 'kirill', 'sql', '_adventureworkslt')

In [ ]:
set_lock(2014, 1, 'sophie', 'sql', '_adventureworkslt')

In [ ]:
remove_lock(2014, 1, 'kirill', 'sql', '_adventureworkslt')

In [ ]:
remove_lock(2014, 1, 'sophie', 'sql', '_adventureworkslt')

In [80]:
accept_plan(2014, 1, 'kirill', 'sql', '_adventureworkslt')

In [81]:
accept_plan(2014, 1, 'sophie', 'sql', '_adventureworkslt')

In [16]:
credentials

[('ivan', 'sql'), ('kirill', 'sql'), ('sophie', 'sql')]

In [37]:
year = 2014
quarter = 1

for idx in range(1, len(credentials)):
    user, pwd = credentials[idx][0], credentials[idx][1]
    set_lock(year=year, quarter=quarter, user=user, pwd=pwd)

In [39]:
year = 2014
quarter = 1

for idx in range(1, len(credentials)):
    user, pwd = credentials[idx][0], credentials[idx][1]
    remove_lock(year=year, quarter=quarter, user=user, pwd=pwd)

In [41]:
year = 2014
quarter = 1

for idx in range(2, 3):
    user, pwd = credentials[idx][0], credentials[idx][1]
    accept_plan(year, quarter, user, pwd)